In [60]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [51]:
txt = 'great product.i like it'

In [61]:
# Function to remove punctuations
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [62]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [63]:
with open('../static/model/corpora/stopwords/english','r')as file:
    sw= file.read().splitlines()

In [64]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [65]:
from nltk.stem import PorterStemmer
ps= PorterStemmer()

In [66]:
def preprocessing(text):
    data = pd.DataFrame(text, columns =['tweet'])
    data["tweet"] = data["tweet"].apply(lambda x: x.lower())  #Convert to lowercase
    data["tweet"] = data["tweet"].apply(lambda x: re.sub(r'https?://\S+', '', x)) # Remove URLs
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data["tweet"].str.replace(r'\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: "".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x: "".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

In [71]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new 

In [72]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    if prediction == 1:
        return 'negative'
    else:
        return 'positive'

In [74]:
def preprocessing(text):
    # Ensure `text` is a list
    if isinstance(text, str):  # If a single string
        text = [text]          # Wrap it in a list
    elif not isinstance(text, (list, tuple)):  # If not iterable
        raise ValueError("Input text must be a string or a list of strings")
    
    # Create DataFrame
    data = pd.DataFrame(text, columns=['tweet'])

    # Apply preprocessing
    data["tweet"] = data["tweet"].apply(lambda x: x.lower())  # Convert to lowercase
    data["tweet"] = data["tweet"].apply(lambda x: re.sub(r'https?://\S+', '', x))  # Remove URLs
    data["tweet"] = data["tweet"].apply(remove_punctuations)  # Remove punctuations

    return data["tweet"]


In [75]:
txt = "awesome product. i love it"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'positive'